<a href="https://colab.research.google.com/github/KimManSub/opencv_practice/blob/main/4_%EA%B0%9D%EC%B2%B4_%EC%B6%94%EC%A0%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 차량 객체추적

## 1. 구글 마운트

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')
# Google Colab 환경에서 Google Drive를 연결하여 파일을 읽거나 저장할 수 있도록 설정.
# '/content/drive' 경로로 드라이브가 마운트됨.

%cd /content/drive/MyDrive/Colab Notebooks
# 현재 작업 디렉토리를 Google Drive의 'Colab Notebooks' 폴더로 변경.
# YOLO 관련 파일이 여기에 저장되거나 불러올 예정이므로 작업 경로를 설정.

## 2. YOLO 모델 파일 다운로드

In [7]:
# YOLO 모델 파일 다운로드 (이미 필요한 파일을 다운로드)

!wget https://pjreddie.com/media/files/yolov3.weights -O /content/yolov3.weights
# YOLO가 어떤 물체를 어떻게 찾아야 하는지 학습한 내용 담겨져 있음. (물체를 찾아내는 방법을 기억 하는파일)

# YOLO의 사전 학습된 가중치 파일(yolov3.weights)을 다운로드.
# - `wget`: URL에서 파일을 다운로드하는 명령어.
# - '-O': 다운로드한 파일을 '/content/yolov3.weights'로 저장.
# 이 파일은 YOLO 모델이 학습된 정보(가중치)를 담고 있어 모델 실행 시 필요.

!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg -O /content/yolov3.cfg
# YOLO가 물체를 찾는 방법을 정의한 설계도, 레이어가 어떻게 생겻는지?,어떤 방법으로 물체를 찾을 지에 대한 규칙 담겨져 있음

# YOLO 모델의 구성 파일(yolov3.cfg)을 다운로드.
# - 이 파일은 YOLO 네트워크 구조를 정의(예: 레이어 수, 크기 등).
# - 다운로드 후 '/content/yolov3.cfg'로 저장.

!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names -O /content/coco.names
# YOLO가 찾을 수있는 물체들의 이름이 들어 있음.

# COCO 데이터셋의 클래스 이름 파일(coco.names)을 다운로드.
# - 각 클래스의 이름(예: 'person', 'car', 'dog')이 저장된 텍스트 파일.
# - YOLO가 탐지한 객체의 레이블(이름)을 출력할 때 사용.
# - 다운로드 후 '/content/coco.names'로 저장.


#만약 coco.names 에 없는 물체를 찾을려고 한다면

# 1. 포트홀 이미지를 많이 모아 모델에 학습

# 2. 라벨링작업

# 3. 학습데이터 준비 (YOLO 형식에 맞게 변환) - 이미지에 대해 객체의 클래스 바운딩 박스 좌표를 포함한 텍스트 파일 필요

# 4. YOLO모델을 사용해서 학습 시작

# 5. 새로운 포트홀 클래스 추가 , yolov3.cfg 파일 클래스 수를 수정, coco.names 파일에도 포트홀 클래스 추가

--2025-01-02 05:51:38--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘/content/yolov3.weights’

/content/yolov3.wei 100%[===================>] 236.52M  19.3MB/s    in 14s     

2025-01-02 05:51:52 (17.5 MB/s) - ‘/content/yolov3.weights’ saved [248007048/248007048]

--2025-01-02 05:51:52--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘/content/yolov3.cfg’

/content/yolov3.cfg 100%[===================>]   8.15K 

## GPU 활성화 확인

In [1]:
import torch
print(torch.cuda.is_available())  # True이면 GPU가 활성화된 상태

True


## YOLO 파일 로드

In [10]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# YOLO 모델(가중치 모델,구조를 정의한 파일) 로드
yolo_net = cv2.dnn.readNet('/content/yolov3.weights', '/content/yolov3.cfg')

# GPU 사용을 위한 설정
yolo_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)  # GPU 백엔드를 CUDA로 설정
yolo_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)  # CUDA를 타겟으로 지정

# YOLO의 레이어 이름 가져오기
layer_names = yolo_net.getLayerNames()
# YOLO 모델의 모든 레이어 이름을 가져오는 부분
# (레이어란? 딥러닝 모델이 데이터를 처리하는 각 단계나 층을 의미함) 예) 입력레이어,분석레이어,출력레이어
# 모델 안에 있는 레이어의 이름을 리스트로 반환 해줌

output_layers = yolo_net.getUnconnectedOutLayersNames()  # 출력 레이어 이름
# YOLO모델 출력 레이어의 이름을 가져옴
# 출력레이어란? 모델의 최종 결과를 생성하는 레이어-- > 객체 탐지 결과(물체가 무엇인지?, 어디에 있는지 등)

# COCO 클래스 라벨 로드
with open('/content/coco.names', 'r') as f: #리스트 컴프리헨션 ([line.strip() for line in f.readlines()])  이렇게하면 모든 줄에 대해 적용해서 모든 클래스 이름을 하나식 정리 한후 새로운 리스트 만들어줌
    classes = [line.strip() for line in f.readlines()]  # COCO 데이터셋 클래스 이름을 리스트 형식으로 로드
# 파일 안에는 YOLO가 탐지 할수 있는 물체들의 이름 저장하고 있음

for class_name in classes:
    print(class_name)

person
bicycle
car
motorbike
aeroplane
bus
train
truck
boat
traffic light
fire hydrant
stop sign
parking meter
bench
bird
cat
dog
horse
sheep
cow
elephant
bear
zebra
giraffe
backpack
umbrella
handbag
tie
suitcase
frisbee
skis
snowboard
sports ball
kite
baseball bat
baseball glove
skateboard
surfboard
tennis racket
bottle
wine glass
cup
fork
knife
spoon
bowl
banana
apple
sandwich
orange
broccoli
carrot
hot dog
pizza
donut
cake
chair
sofa
pottedplant
bed
diningtable
toilet
tvmonitor
laptop
mouse
remote
keyboard
cell phone
microwave
oven
toaster
sink
refrigerator
book
clock
vase
scissors
teddy bear
hair drier
toothbrush


## 차량 검출 함수 정의

In [ ]:
def detect_vehicles(frame):
    # 입력 프레임의 크기 정보 가져오기
    height, width, channels = frame.shape #프레임 세로,가로,이미지 채널수

    # YOLO 입력 데이터 변환
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    # 입력된 이미지 프레임,
    #이미지 픽셀값을 0과 1사이로 정규화 하는값(YOLO가 사용하는 값-->0.0392),
    #YOLO 모델에 맞는 입력 이미지크기(416,416),(이미지 평균 색상값),
    #이미지의 색상을 RGB로유지
    #이미지 자르지 않음
    yolo_net.setInput(blob)  # 변환된 데이터를 YOLO 네트워크에 입력
    outs = yolo_net.forward(output_layers)  # 네트워크 출력

    # 검출 결과 저장용 리스트
    class_ids = []  # 클래스 ID(탐지된 객체의 클래스 ID 저장용도하는 리스트)
    confidences = []  # 각 객체의 신뢰도(각 객체의 신뢰도 값 저장하는 리스트)
    boxes = []  # 객체의 경계 상자 좌표(객체의 경계 상자 좌표를 저장하는 리스트)

    for out in outs:
        for detection in out:
            scores = detection[5:]  # 클래스별 신뢰도 점수
            class_id = np.argmax(scores)  # 가장 높은 점수의 클래스 ID 선택
            confidence = scores[class_id]  # 해당 클래스의 신뢰도

            # 신뢰도 50% 이상이고, 클래스가 '자동차(2)'일 경우
            if confidence > 0.5 and class_id == 2:
                # 중심 좌표와 박스 크기 계산
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # 왼쪽 상단 좌표 계산
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])  # 경계 상자 좌표 저장
                confidences.append(float(confidence))  # 신뢰도 저장
                class_ids.append(class_id)  # 클래스 ID 저장

    return boxes, confidences, class_ids
#각각 계산된 박스좌표,신뢰도, 클래스 아이디를 리스트 형식으로 저장

## 메인 함수 정의

In [6]:
#메인 함수 진행순서

#1.입력
# 비디오 파일 읽기
# 출력 비디오 파일 설정

#2. 처리
# 프레임 단위로 처리
# 차량 검출 및 추적

#3. 저장
# 출력 비디오 결과에 저장
# 비디오 파일 종료 및 다운로드 링크 제공

def main():
    # 입력 비디오 파일 경로 및 출력 비디오 파일 경로
    video_path = "/content/drive/MyDrive/Colab Notebooks/이미지처리/car.mp4"
    output_path = "/content/drive/MyDrive/Colab Notebooks/car_tracking_output.mp4"

    # 비디오 파일 읽기
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: 비디오 파일을 열 수 없습니다.")
        return

    # 입력 비디오의 속성 가져오기
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 5  # 초당 프레임 수 (그렇다고 너무 적게 잡으면 차량 검출이 안됨)

    # 출력 비디오의 해상도와 코덱 설정
    resized_width = 640
    resized_height = 480
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_path, fourcc, fps, (resized_width, resized_height))

    # 추적기 관련 변수 초기화
    trackers = []  # [추적기 객체, 실패 횟수] 저장 (차량 하나에 대한 추적 담당)
    detection_interval = 5  # 몇 프레임마다 YOLO를 사용해 재검출할지 (5프레임)
    max_tracking_failures = 3  # 추적 실패 허용 횟수 ()

    frame_count = 0  # 현재 프레임 번호
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % (30 // 5) != 0:  # 프레임 간 샘플링
            continue

        frame = cv2.resize(frame, (resized_width, resized_height))  # 프레임 크기 조정

        if frame_count % detection_interval == 0 or len(trackers) == 0:
            # YOLO를 사용하여 차량 검출
            boxes, confidences, class_ids = detect_vehicles(frame)
            indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

            # 새로운 추적기 초기화
            trackers = []
            if len(indices) > 0:
                for i in indices.flatten():
                    x, y, w, h = boxes[i]
                    tracker = cv2.TrackerCSRT_create()  # CSRT 기반 추적기 생성
                    tracker.init(frame, (x, y, w, h))
                    trackers.append([tracker, 0])  # 추적기와 실패 횟수를 함께 저장

        else:
            # 추적기 업데이트
            updated_trackers = []
            for tracker, fail_count in trackers:
                success, bbox = tracker.update(frame)
                if success:
                    x, y, w, h = [int(v) for v in bbox]
                    if (x < 0 or y < 0 or x + w > resized_width or y + h > resized_height or w * h < 100):
                        continue
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, "Car", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                    updated_trackers.append([tracker, 0])  # 성공한 경우 실패 횟수를 초기화
                else:
                    fail_count += 1
                    if fail_count < max_tracking_failures:
                        updated_trackers.append([tracker, fail_count])

            trackers = updated_trackers

        out.write(frame)  # 결과 프레임을 출력 비디오에 저장

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("차량 추적 완료")

    # Colab에 다운로드 링크 제공
    from google.colab import files
    files.download(output_path)

차량 추적 완료


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 메인 함수 실행

In [ ]:
# 메인 함수 실행
if __name__ == "__main__":
    main()